In [ ]:
import json
import random
from pathlib import Path
from typing import Iterable, Dict, Union

import torch
from tsum import tsum
from ndtools import staged_max_flow as smf

import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Randomly generate probabilities

In [ ]:
def rewrite_probs_json(
    in_path: Union[str, Path],
    out_path: Union[str, Path],
    *,
    low: float = 0.01,
    high: float = 0.1,
    seed: int | None = None,
) -> dict:
    """
    Load an existing probs.json and replace:
      - probs[x]["0"]["p"] ~ Uniform(low, high)
      - probs[x]["1"]["p"] = 1 - probs[x]["0"]["p"]

    All other fields and schema are preserved.
    """

    if seed is not None:
        random.seed(seed)

    in_path = Path(in_path)
    out_path = Path(out_path)

    with in_path.open("r", encoding="utf-8") as f:
        probs = json.load(f)

    for x, states in probs.items():
        if "0" not in states or "1" not in states:
            raise KeyError(f"{x} does not have states '0' and '1'")

        p0 = random.uniform(low, high)
        p1 = 1.0 - p0

        states["0"]["p"] = p0
        states["1"]["p"] = p1

    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as f:
        json.dump(probs, f, indent=4)

    return probs

In [ ]:
DATASET = Path(r"data") 

rewrite_probs_json(
    in_path=DATASET / "probs.json",
    out_path=DATASET / "probs_random.json",
    low=0.01,
    high=0.1,
    seed=42,
)

{'x1': {'0': {'p': 0.06754841186120954, 'remaining_capacity_ratio': 0.0},
  '1': {'p': 0.9324515881387905, 'remaining_capacity_ratio': 1.0}},
 'x2': {'0': {'p': 0.012250967970040025, 'remaining_capacity_ratio': 0.0},
  '1': {'p': 0.98774903202996, 'remaining_capacity_ratio': 1.0}},
 'x3': {'0': {'p': 0.03475263865322074, 'remaining_capacity_ratio': 0.0},
  '1': {'p': 0.9652473613467792, 'remaining_capacity_ratio': 1.0}},
 'x4': {'0': {'p': 0.03008896643339405, 'remaining_capacity_ratio': 0.0},
  '1': {'p': 0.9699110335666059, 'remaining_capacity_ratio': 1.0}},
 'x5': {'0': {'p': 0.07628240927476111, 'remaining_capacity_ratio': 0.0},
  '1': {'p': 0.9237175907252388, 'remaining_capacity_ratio': 1.0}},
 'x6': {'0': {'p': 0.07090295386806203, 'remaining_capacity_ratio': 0.0},
  '1': {'p': 0.929097046131938, 'remaining_capacity_ratio': 1.0}},
 'x7': {'0': {'p': 0.09029616109343609, 'remaining_capacity_ratio': 0.0},
  '1': {'p': 0.9097038389065639, 'remaining_capacity_ratio': 1.0}},
 'x8': {

# Load data

Basic information.

In [ ]:
DATASET = Path(r"data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_random.json").read_text(encoding="utf-8"))

def s_fun(comps_st):
    flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, nodes, edges, probs_dict, target_flow = 0.5 )

    sys_st = 1 if sys_st_str == 's' else 0
    return flow, sys_st, None

row_names = list(probs_dict.keys())
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)


TSUM results

In [19]:
TSUMPATH = Path("tsum_res")

sys_surv_st = 1  # system survival state

with open(TSUMPATH / "rules_geq_1.json", "r") as f:
    rules_surv = json.load(f)
with open(TSUMPATH / "rules_leq_0.json", "r") as f:
    rules_fail = json.load(f)

rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)


Get a little more rules for the changed probabilities.

In [21]:
result = tsum.run_rule_extraction_by_mcs(
    # Problem-specific callables / data
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    # Existing rules
    rules_surv = rules_surv,
    rules_fail = rules_fail,
    rules_mat_surv=rules_mat_surv,
    rules_mat_fail=rules_mat_fail,
    # TSUM parameters
    n_state=n_state,
    sys_surv_st=sys_surv_st,
    unk_prob_thres = 1e-6,
    output_dir="tsum_res_random"
) 

---
Round: 1, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 65, Survival rules: 34, Failure rules: 31
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 1, System value: 0.5. Total samples: 100000.
New rule (No. of conditions: 23): {'x2': ('>=', 1), 'x14': ('>=', 1), 'x17': ('>=', 1), 'x28': ('>=', 1), 'x30': ('>=', 1), 'x32': ('>=', 1), 'x33': ('>=', 1), 'x35': ('>=', 1), 'x46': ('>=', 1), 'x51': ('>=', 1), 'x53': ('>=', 1), 'x55': ('>=', 1), 'x57': ('>=', 1), 'x58': ('>=', 1), 'x61': ('>=', 1), 'x62': ('>=', 1), 'x64': ('>=', 1), 'x65': ('>=', 1), 'x66': ('>=', 1), 'x82': ('>=', 1), 'x84': ('>=', 1), 'x85': ('>=', 1), 'x87': ('>=', 1), 'sys': ('>=', 1)}
Updated sys_vals: [0.5]
---
Round: 2, Unk. prob.: 2.000e-05
Surv probs: 6.149e-01, Fail probs: 3.851e-01
No. of non-dominant rules: 66, Survival rules: 35, Failure rules: 31
Survival sample found from sampling.
No. of existing rules remov

In [23]:
TSUMPATH = Path("tsum_res_random")

sys_surv_st = 1  # system survival state

rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

# Birnbaum's measure

In [25]:
bms_dict = {}

for x in probs_dict.keys():
    print(f"Calculating BM for {x}..")
    
    ps1_x1 = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun,
        sys_surv_st = sys_surv_st,
        n_sample = 10_000_000
    )

    ps1_x0 = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 0},
        row_names = row_names,
        s_fun = s_fun,
        sys_surv_st = sys_surv_st,
        n_sample = 1_000_000
    )

    bm_x = max([0, ps1_x1['survival'] - ps1_x0['survival']])
    bms_dict[x] = bm_x

    print(f"BM({x}) = {bm_x:.3e}\n")

out_path = TSUMPATH / "bms_random.json"
with open(out_path, "w") as f:
    json.dump(bms_dict, f, indent=4)

Calculating BM for x1..
BM(x1) = 4.936e-02

Calculating BM for x2..
BM(x2) = 8.422e-02

Calculating BM for x3..
BM(x3) = 3.860e-04

Calculating BM for x4..
BM(x4) = 0.000e+00

Calculating BM for x5..
BM(x5) = 2.667e-04

Calculating BM for x6..
BM(x6) = 3.086e-04

Calculating BM for x7..
BM(x7) = 7.830e-04

Calculating BM for x8..
BM(x8) = 0.000e+00

Calculating BM for x9..
BM(x9) = 0.000e+00

Calculating BM for x10..
BM(x10) = 0.000e+00

Calculating BM for x11..
BM(x11) = 9.640e-05

Calculating BM for x12..
BM(x12) = 0.000e+00

Calculating BM for x13..
BM(x13) = 0.000e+00

Calculating BM for x14..
BM(x14) = 0.000e+00

Calculating BM for x15..
BM(x15) = 9.679e-04

Calculating BM for x16..
BM(x16) = 9.253e-04

Calculating BM for x17..
BM(x17) = 3.476e-04

Calculating BM for x18..
BM(x18) = 6.540e-04

Calculating BM for x19..
BM(x19) = 2.421e-04

Calculating BM for x20..
BM(x20) = 5.248e-04

Calculating BM for x21..
BM(x21) = 0.000e+00

Calculating BM for x22..
BM(x22) = 3.480e-04

Calcul